In [6]:
unknown_rule, rest_unknown_list = UNK.remove_no_meanings(unknown_list, mecab)
len(rest_unknown_list)

7836

In [7]:
post_ma_list = UNK.shape_changer(unknown_rule)

rule 기록하기

In [8]:
post_ma = Post_ma(tokenizer_name='mecab')

Load Complete! total 7110 rules


In [10]:
# post_ma.update(post_ma_list, write=True)

In [11]:
post_ma.load()

In [9]:
train_ma_docs = post_ma.replace_ma_docs(train_ma_docs, UNK.unknown_loc)

부호 빼고 사전을 만들어보자!

In [138]:
UNK = Unknown_words(train_ma_docs, app_id_list)
unknown_list = UNK.get_unknown_words()

Unknown(중복제거): 14946, Total Unknown: 97512, Total MA: 10679387


In [139]:
unknown_list[20:40]

['ㅠ제발요ㅠㅠ',
 'ㅠㅠㅜ',
 '냑',
 '쳣캏',
 'ㅜㅜ이럴줄알았으면',
 'ㅠㅠ영어',
 'ㅠㅠㅠㅠ스티커',
 '페메영통할때',
 '핮',
 '쬬',
 'ㅌㄴ',
 '렇게좋다고알바들떠드는지ㅡ',
 '즁에는',
 '귯걸',
 '뎃히니주세여ㅠ',
 'ㅜㅠ버전도',
 'ㅗ채',
 'ㅜㅜ데이터',
 '얗쟈ㅗ재누ㅏ유ㅐㅓ두애',
 '썻는대요']

In [142]:
UNK.unknown_loc['페메영통할때']

[['454638411', (483, 6)]]

In [181]:
def choice(train_docs, app_id_list):
    li = []
    for app_id in app_id_list:
        for i, doc in enumerate(train_docs[app_id]['reviews']):
            if (' ' not in doc) & (len(doc)>=50):
                li.append([app_id, i])
    idx = np.random.randint(0, len(li)-1)
    a = li[idx]
    return train_docs[a[0]]['reviews'][a[1]], a

In [183]:
test2, a = choice(train_docs, app_id_list)
test2, a

('안드로이드쓸때캔디카메라가무음카메라여서썻는데어쩐일인지아이폰6로버꿔서캔디카메라를쓰는데무음이안되네여답변좀부탁드려여....ㅠㅠ',
 ['881267423', 454])

In [184]:
mecab.pos(test2)

[('안드로이드', 'NNG'),
 ('쓸', 'VV+ETM'),
 ('때', 'NNG'),
 ('캔디', 'NNG'),
 ('카메라', 'NNG'),
 ('가', 'JKS'),
 ('무음', 'NNG'),
 ('카메라', 'NNG'),
 ('여서', 'VCP+EC'),
 ('썻는데어쩐일인지아이폰', 'UNKNOWN'),
 ('6', 'SN'),
 ('로버', 'NNG'),
 ('꿔', 'VV+EC'),
 ('서', 'VV+EC'),
 ('캔디', 'NNG'),
 ('카메라', 'NNG'),
 ('를', 'JKO'),
 ('쓰', 'VV'),
 ('는데', 'EC'),
 ('무음', 'NNG'),
 ('이', 'JKS'),
 ('안', 'MAG'),
 ('되', 'VV'),
 ('네여', 'EF'),
 ('답변', 'NNG'),
 ('좀', 'MAG'),
 ('부탁', 'NNG'),
 ('드려', 'VV+EF'),
 ('여', 'JKS+EF'),
 ('.', 'SF'),
 ('...', 'SY'),
 ('ㅠㅠ', 'UNKNOWN')]

In [186]:
mecab.pos('썻는데 어쩐일인지아이폰')

[('썻는데', 'UNKNOWN'),
 ('어쩐', 'VV+ETM'),
 ('일', 'NNG'),
 ('인지', 'VCP+EC'),
 ('아이폰', 'NNP')]

---

test데이터 생성해보기

In [3]:
train_docs, app_id_list = get_data_json('./data/train_json.txt')

In [4]:
test = train_docs['333903271']['reviews'][460]
test

'유령계정이 자꾸 되네요... 뭐 의심갈만한거 있지도 않은데 계속해서 유령되요 제발 이것좀 어떻게 해주세요ㅜㅜㅜㅜㅜ진짜 찾아갈꺼에요ㅜㅜㅜ'

In [5]:
test2 = train_docs['417608082']['reviews'][373]
test2

'잘되다가갑자기경유노선목록이제공되지안다고하고노선정보도확인안되고뭡니까?확인해주세요~저는아이폰을사용중입니다'

---

---

## 구상:
우선 형태소 분석함 > 형태소 분석이 안된 구문들 추출함 > 안되는 이유 체크(띄어쓰기에 반영) + 테스트할 단어 교정 체크 > 



단어 추출함 [명사, 형용사, 동사] (2글자 이상) / [조사등] 추출 > 

그리고 다시 형태소 분석전부터 시작해서 띄어쓰기 교정를 우선 시행 후, 교정할 단어를 교정하고(normalize) 형태소 분석

1) 그냥 형태소 분석 때림

In [2]:
from collections import Counter, defaultdict
from konlpy.tag import Mecab, Komoran
from utils import Unknown_words, Post_ma, Make_dictionay
import ujson
mecab = Mecab()
komoran = Komoran()
def get_data_json(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        texts = file.read()
        data = ujson.loads(texts)
        app_ids_list = list(data.keys())
    return data, app_ids_list

In [7]:
train_ma_docs, app_id_list = get_data_json('./data/ma_train_json_mecab.txt')

2) 안된 이유 찾기:

첫째로, 자음, 모음 단독으로 붙여진 것들은 형태소 분석이 안됨.

둘째로, mecab이 분석 못하는 단어들 (이건 솔직히 background지식): 축약어, 오타 등등

셋째로, 띄어쓰기가 제대로 안되어 있으면 형태소 분석이 안됨

In [20]:
UNK = Unknown_words(train_ma_docs, app_id_list)
unknown_list = UNK.get_unknown_words()

Unknown(중복제거): 14946, Total Unknown: 97512, Total MA: 10679387


In [127]:
n = np.random.randint(0, len(unknown_list)-10)
unknown_list[n:(n+10)]

['ㅠㅠㅠ다',
 '앜ㅋㅋ',
 '튕겻네요',
 '괜찬ㄹ은어플인거같아요',
 '짦은',
 '뎃받게하지말구여',
 '밌다ㅠㅜ',
 '낞고',
 'ㅠㅠㅠㅠ공부하던거',
 'ㅡ설명이없에']

따라서, 아래와 같은 rule을 만듬

행태소 분석이 이상하게 된 경우:
1. 교정하고 싶은 단어를 찾아서 우선 교정해준다(보수적으로 하기!, 주로 축약어를 진행)
1. 단어 유사도를 측정하여 오타를 잡아보자!(대표적인 오타들을 조정? 됫 > 됐)


형태소 분석이 아예 안된 경우:
1. 단일 자음 모음은 첫 번째와 그 마지막 번째를 띄워주기
1. 형식 형태소 옆에 띄어쓰기 해주기 


### 형식 형태소가 아예 안된 경우

* 단일 자음 모음

첫번째와 그 마지막 번째를 골라서 묶어주고 양옆에는 띄어쓰기 해주기

In [101]:
# 나중에 단어 유사도를 측정하기 위해서 만들어 놓음
chosung_list = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
jungsung_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ',
                 'ㅢ', 'ㅣ']
jongsung_list = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ',
                 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

In [1]:
from itertools import islice
from collections import deque

In [102]:
def space_jamo(doc):
    doc_splited = [char for char in doc]
    space_idx = get_spacing_index(doc_splited)
    tokens_list = split_doc_to_words(doc, space_idx)
    new_text = ' '.join(tokens_list)
    return new_text

def split_doc_to_words(doc, space_idx):
    tokens_list = []
    for i in range(2, len(space_idx)+1):
        start, stop = deque(islice(space_idx, i), maxlen=2)
        tokens_list.append(doc[start:stop])
    return tokens_list

def get_spacing_index(doc_splited):
    space_idx = []
    doc_len = len(doc_splited)
    for i in range(1, doc_len):
        window = deque(islice(doc_splited, i), maxlen=2)  # 번째 원소 전까지 window가 움직이는거 
        if check_jamo(window):
            space_idx.append(i-1)
    # slicing위한 작업
    if 0 not in space_idx:
        space_idx.insert(0, 0)
    if doc_len not in space_idx:
        space_idx.insert(len(space_idx), doc_len)
    return space_idx
   
def check_jamo(window):
    check_list = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 
                  'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', 'ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 
                  'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ', 'ㄳ', 'ㄵ', 
                  'ㄶ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅄ']
    mask = [True if char in check_list else False for char in window]
    if sum(mask) == 1:
        return True
    else:
        return False

In [105]:
test

'유령계정이 자꾸 되네요... 뭐 의심갈만한거 있지도 않은데 계속해서 유령되요 제발 이것좀 어떻게 해주세요ㅜㅜㅜㅜㅜ진짜 찾아갈꺼에요ㅜㅜㅜ'

In [106]:
space_jamo(test)

'유령계정이 자꾸 되네요... 뭐 의심갈만한거 있지도 않은데 계속해서 유령되요 제발 이것좀 어떻게 해주세요 ㅜㅜㅜㅜㅜ 진짜 찾아갈꺼에요 ㅜㅜㅜ'

* 형식 형태소 오른쪽에 띄어쓰기 해주기

unknown생겼을 때 하면 효과가 좋을 듯

1) 사전 만들기

전부 형태소 분석이 불가 한 것들은 우선 NULL 처리함, 그리고 2자 이상만 취급한다 

[위키백과:형태소](https://ko.wikipedia.org/wiki/%ED%98%95%ED%83%9C%EC%86%8C)

In [10]:
# 실질 형태소: 주로 왼쪽
full_morphemes = ['NNG','NNP','NNB','NNBC','NR','NP','VV','VA','VX','VCP','VCN','MM',
                 'MAG','MAJ','IC']
# 형식 형태소: 주로 오른쪽
empty_morphemes = ['JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JX','JC','EP','EF',
                  'EC','ETN','ETM','XPN','XSN','XSV','XSA'] 
# 'XPN'(접두사) 은 띄어쓰기 제외, ETM중 (-기)만 띄어쓰기함

# 한자, 숫자, 영어 등
not_koreans = ['SL','SH','SN']
# 부호들
signs = ['SF', 'SE', 'SSO', 'SSC', 'SC', 'SY']

In [11]:
make_dict = Make_dictionay()

In [13]:
make_dict.fit(train_ma_docs, 
              delete_pos=not_koreans+signs,
              min_word_length=2,
              maxwords=100000)

total words: 51759
complete!


형식 형태소를 추출해서 그 오른쪽에 띄어쓰기를 진행한다. 무조건 2글자 ...한 글자는 안댐

In [14]:
def get_EM_list(dictionary):
    # 'EP'(선어말어미), 'XPN'(접두사) 은 띄어쓰기 제외, ETM중 (+기)만 띄어쓰기함
    empty_morphemes = ['JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JX','JC','EF',
                       'EC','ETN','ETM','XSN','XSV','XSA'] 
    full_morphemes = ['NNG','NNP','NNB','NNBC','NR','NP','VV','VA','VX','VCP','VCN','MM',
                      'MAG','MAJ','IC']
    EM_list = []
    for k in dictionary.keys():
        if k == 'NULL':
            continue
        word, morph = k.split('/')
        if len(morph.split('+')) > 1:
            if check_empty_morph(word, morph.split('+'), empty_morphemes):
                EM_list.append(word)
        elif morph in empty_morphemes:
            EM_list.append(word)
            
    return list(set(EM_list))
            
def check_empty_morph(word, morph, empty_morphemes):
    for m in morph:
         if m in empty_morphemes:
            if (m == 'ETN') & (word == '기'):  # ETN은 뒤에 띄어쓰기를 안하나 +기 는 해야됨
                return True
            elif (m == 'ETN') & (word != '기'):
                return False
            else:
                return True
            

In [15]:
EM_list = get_EM_list(make_dict.word_dict_idx)

In [16]:
EM_list[:10] # 조금 문제가 있음 어쨌든 해봅시다

['처진다', '걸텐데', '굳힌', '서도', '고우나', '떠올려서', '다릅니다', '언데', '그른가', '써야할지']

만약에 이것들 뒤로 spacing이 되어 있으면 space안함

In [70]:
def space_EMs(doc, EM_list):
    n = len(EM_list)
    i = 0
    while i < n:
        empty_morpheme = EM_list[i]
        doc = space_empty_morpheme(doc, empty_morpheme)
        i += 1
    return doc

def space_empty_morpheme(doc, empty_morpheme):
    assert doc != '' 
    if empty_morpheme in doc:
        start_idx = doc.index(empty_morpheme)
        stop_idx = start_idx + len(empty_morpheme)
        doc = doc[:stop_idx] + ' ' + doc[stop_idx:]
        return doc
    else:
        return doc

In [72]:
aa = space_EMs(test2, EM_list)
aa

'잘되다가 갑자기경유노선목록이제공되지안다 고  하고 노선정보도확인 안되고뭡니까  ?확인해 주세요 ~저는아이폰을사용중입니다  '

In [50]:
test2

'잘되다가갑자기경유노선목록이제공되지안다고하고노선정보도확인안되고뭡니까?확인해주세요~저는아이폰을사용중입니다'

---

안되는 구문 추출 해서 적용하자!

In [3]:
UNK = Unknown_words(train_ma_docs, app_id_list)

In [4]:
unknown_list = UNK.get_unknown_words()

Unknown(중복제거): 14946, Total Unknown: 97512, Total MA: 10679387


In [99]:
mecab.pos('빙신드드라') 

[('빙신', 'NNG'), ('드', 'VV'), ('드라', 'EC')]

교정 할 단어들: (그냥 품사 안하고 원글에서 교정할 수 도 있음, 생각좀 해보자)

/data/preprocessing 저장해둠 